In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import sys


from bokeh.io import show, output_file, output_notebook
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, StaticLayoutProvider
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from scipy.io import loadmat

output_notebook()
%matplotlib inline

Loading BokehJS ...

In [2]:
connectivity = loadmat("../data/aux_meanACS69.mat")['meanACS']
region_names = pd.read_csv('../data/dk_names.csv')['Atlas']
coords = pd.read_csv('../data/dk.csv').set_index('Name')

conn = pd.DataFrame(connectivity)
conn.columns = list(region_names)
conn.index = list(region_names)

In [3]:
for region in region_names:
    if region not in coords.index:
        conn = conn.drop(index=region, columns=region)

# clip out any connectivity below cutoff (e.g. 2)
cutoff = 2.
conn = conn.clip(lower=cutoff).replace(cutoff, 0)

coords = coords.sort_values(by=['hemisphere','lobe'])

In [4]:
# assign coordinates to each region, first in the top, then bottom. Always from (0,0)
radius = 1.0
positions = {}
shift = 2*np.pi/136
for i in range(68):
    region = coords.index[i]
    if i < 34:
        x = radius*np.cos(2*np.pi*i/68 + np.pi/2. + shift)
        y = radius*np.sin(2*np.pi*i/68 + np.pi/2. + shift)
    else:
        i = i - 33
        x = radius*np.cos(-2*np.pi*i/68 + np.pi/2. + shift)
        y = radius*np.sin(-2*np.pi*i/68 + np.pi/2. + shift)
    positions.update({region:[x,y]})

In [10]:
G = nx.from_pandas_adjacency(conn)

plot = Plot(plot_width=500, plot_height=900,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
plot.title.text = "Brain Connectivity - DK Atlas"

plot.add_tools(HoverTool(tooltips=[("", "@name")]), TapTool(), BoxSelectTool())

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size=12, fill_color=Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=3)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=3)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=3)

graph_renderer.selection_policy = NodesAndLinkedEdges()
# graph_renderer.inspection_policy = EdgesAndLinkedNodes()

fixed_layout_provider = StaticLayoutProvider(graph_layout=positions)
graph_renderer.layout_provider = fixed_layout_provider
graph_renderer.node_renderer.data_source.data.update({'name':graph_renderer.node_renderer.data_source.data['index']})
# graph_renderer.node_renderer.data_source.data.update({'nl':neighbor_list})

plot.renderers.append(graph_renderer)

output_notebook()
show(plot)
# output_file("interactive_graphs.html")

Loading BokehJS ...

In [8]:
# neighbor_list = []
# for region in graph_renderer.node_renderer.data_source.data['index']:
#     neighbor_list.append([n for n in G.neighbors(region)])

### todo:
- right vs left
- color nodes by lobe
- add lobe legend
- highlight nodes with connection
- add name to nodes with connection
- add [Bezier](https://bokeh.pydata.org/en/latest/docs/user_guide/graph.html)